In [42]:
import requests
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import time
import datetime as dt

print('Imports completed!')

Imports completed!


# TEAMS

In [43]:
url = "https://api-nba-v1.p.rapidapi.com/teams"
headers = {
	"x-rapidapi-key": "dda6cf782dmshebe3119a485b548p154863jsn7c9c31f1e7a7",
	"x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
}

# Extract from url and header parameters
def extract_data(api_url, headers):
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API request failed with status code: {response.status_code}")

# Clean the data
def transform_data(raw_data):
    # ID's to not be included
    exclude_ids = {37, 34, 39, 102, 103, 42, 35, 36, 49}

    # Gather Data for Teams
    teams = [
        {
            'team_id': row['id'],
            'team_code': row['code'],
            'team_name': row['name']
        }
        for row in raw_data['response']
        if 'leagues' in row and 'standard' in row['leagues']
        and row['leagues']['standard']['division'] is not None
        and row['id'] not in exclude_ids
    ]
    
    teams_df = pd.DataFrame(teams)
    return teams_df

# Load data into a table
def load_data(df, table_name, connection_string):
    engine = create_engine(connection_string)
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f'Data has been loaded and added to the {table_name} database')

In [44]:
# Run data pipeline
def run_pipeline():
    raw_data = extract_data(url, headers)
    clean_data = transform_data(raw_data)
    connection_string = 'sqlite:///teams_data.db'
    load_data(clean_data, 'teams', connection_string)
    
run_pipeline()

Data has been loaded and added to the teams database


# GAME SCORES

In [45]:
import pandas as pd
from sqlalchemy import create_engine

def read_data(connection_string):
    engine = create_engine(connection_string)
    query = "SELECT * FROM teams"
    df = pd.read_sql(query, con=engine)
    return df

# Read the data back from the database
connection_string = 'sqlite:///teams_data.db'
teams_data = read_data(connection_string)

# Display the data
teams_data

,team_id,team_code,team_name
0,1,ATL,Atlanta Hawks
1,2,BOS,Boston Celtics
2,4,BKN,Brooklyn Nets
3,5,CHA,Charlotte Hornets
4,6,CHI,Chicago Bulls
5,7,CLE,Cleveland Cavaliers
6,8,DAL,Dallas Mavericks
7,9,DEN,Denver Nuggets
8,10,DET,Detroit Pistons
9,11,GSW,Golden State Warriors


In [46]:
from itertools import permutations 
import time

url = "https://api-nba-v1.p.rapidapi.com/games"
headers = {
	"x-rapidapi-key": "dda6cf782dmshebe3119a485b548p154863jsn7c9c31f1e7a7",
	"x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
}

# Get a list of all team ids (only take for 2 for example)
team_ids = teams_data['team_id'].values

def game_matches(team_ids, url, headers):
    # Use team ids to make game matches
    matchups = list(permutations(team_ids, 2))
    
    # Filter games where (id1, id2) != (id2, id1)
    unique_matchups = []
    for game in matchups:
        if game[1] > game[0]:
            unique_matchups.append((game[0], game[1]))
        else:
            continue
    
    # Create an array to store the information
    response_arr = []
    for game in unique_matchups:
        querystring = {"h2h":f"{game[0]}-{game[1]}"}
        print(f"Gathering data for matchup {game[0]}-{game[1]}")
        
        # Make the API request
        response = requests.get(url, headers=headers, params=querystring)
        
        # Check if the response is successful:
        if response.status_code == 200:
            game_data = response.json()
            response_arr.append(game_data)
        else:
            print(f"Failed to gather data for {game[0]}-{game[1]}. Status code: {response.status_code}")
        
        # Avoid hitting rate limits
        time.sleep(1)

    # Display gathered data
    return response_arr

results = game_matches(team_ids=team_ids, url=url, headers=headers)

Gathering data for matchup 1-2
Gathering data for matchup 1-4
Gathering data for matchup 1-5
Gathering data for matchup 1-6
Gathering data for matchup 1-7
Gathering data for matchup 1-8
Gathering data for matchup 1-9
Gathering data for matchup 1-10
Gathering data for matchup 1-11
Gathering data for matchup 1-14
Failed to gather data for 1-14. Status code: 429
Gathering data for matchup 1-15
Failed to gather data for 1-15. Status code: 429
Gathering data for matchup 1-16
Failed to gather data for 1-16. Status code: 429
Gathering data for matchup 1-17
Failed to gather data for 1-17. Status code: 429
Gathering data for matchup 1-19
Failed to gather data for 1-19. Status code: 429
Gathering data for matchup 1-20
Failed to gather data for 1-20. Status code: 429
Gathering data for matchup 1-21
Failed to gather data for 1-21. Status code: 429
Gathering data for matchup 1-22
Failed to gather data for 1-22. Status code: 429
Gathering data for matchup 1-23
Failed to gather data for 1-23. Status 

In [51]:
game_scores = []

for info in results:
    for game in info['response']:
        # Extract date of the game
        date = game.get('date', {}).get('start','N/A')
        
        # Extract teams info
        visitor = game.get('teams', {}).get('visitors', {}).get('name', 'Unknown')
        home = game.get('teams', {}).get('home', {}).get('name', 'Unknown')
     
        visitor_code = game.get('teams', {}).get('visitors', {}).get('code', 'N/A')
        home_code = game.get('teams', {}).get('home', {}).get('code', 'N/A') 
        
        # Initialize lists to store total scores
        visitor_total_scores = []
        home_total_scores = []
        
        # Extract scores by quarter
        visitor_scores = game.get('scores', {}).get('visitors', {}).get('linescore', [])
        home_scores = game.get('scores', {}).get('home', {}).get('linescore', [])
        
        visitor_points = game.get('scores', {}).get('visitors', {}).get('points', '0')
        home_points = game.get('scores', {}).get('home', {}).get('points', '0')
        
        if visitor_scores and home_scores and visitor_points is not None and home_points is not None:
            visitor_total_scores.append(visitor_points)
            home_total_scores.append(home_points)
        
        # Display data if scores and points exist
        if visitor_scores and home_scores and visitor_total_scores and home_total_scores:
            print(f"Date: {date}")
            print(f"Visitor Team: {visitor} ({visitor_code}) - Scores: {visitor_scores} - Total Points: {visitor_total_scores[0]}")
            print(f"Home Team: {home} ({home_code}) - Scores: {home_scores} - Total Points: {home_total_scores[0]}")
            print('-------------------------------------------')
            
        game_scores.append({
            'date': date,
            'visitor': visitor,
            'home': home,
            'visitor_points': visitor_points,
            'home_points': home_points
        })
        
    game_score_df = pd.DataFrame(game_scores)
game_score_df

Date: 2015-11-25T01:00:00.000Z
Visitor Team: Boston Celtics (BOS) - Scores: ['28', '20', '25', '24'] - Total Points: 97
Home Team: Atlanta Hawks (ATL) - Scores: ['33', '24', '24', '40'] - Total Points: 121
-------------------------------------------
Date: 2016-04-09T23:30:00.000Z
Visitor Team: Boston Celtics (BOS) - Scores: ['32', '39', '20', '16'] - Total Points: 107
Home Team: Atlanta Hawks (ATL) - Scores: ['34', '33', '26', '25'] - Total Points: 118
-------------------------------------------
Date: 2016-04-16T23:00:00.000Z
Visitor Team: Boston Celtics (BOS) - Scores: ['19', '15', '31', '36'] - Total Points: 101
Home Team: Atlanta Hawks (ATL) - Scores: ['30', '21', '21', '30'] - Total Points: 102
-------------------------------------------
Date: 2016-04-19T23:00:00.000Z
Visitor Team: Boston Celtics (BOS) - Scores: ['7', '21', '18', '26'] - Total Points: 72
Home Team: Atlanta Hawks (ATL) - Scores: ['24', '19', '18', '28'] - Total Points: 89
-------------------------------------------


,date,visitor,home,visitor_points,home_points
0,2015-11-25T01:00:00.000Z,Boston Celtics,Atlanta Hawks,97.0,121.0
1,2016-04-09T23:30:00.000Z,Boston Celtics,Atlanta Hawks,107.0,118.0
2,2016-04-16T23:00:00.000Z,Boston Celtics,Atlanta Hawks,101.0,102.0
3,2016-04-19T23:00:00.000Z,Boston Celtics,Atlanta Hawks,72.0,89.0
4,2016-04-27T00:30:00.000Z,Boston Celtics,Atlanta Hawks,83.0,110.0
...,...,...,...,...,...
3390,2023-12-28T00:00:00.000Z,Toronto Raptors,Washington Wizards,132.0,102.0
3391,2024-03-24T00:00:00.000Z,Toronto Raptors,Washington Wizards,109.0,112.0
3392,2024-10-11T23:00:00.000Z,Toronto Raptors,Washington Wizards,95.0,113.0
3393,2025-01-30T00:00:00.000Z,Toronto Raptors,Washington Wizards,NaN,NaN
